# companies

> Set of functions for dealing with Copper companies

In [ ]:
#| default_exp companies

In [9]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from cu_api import core, config
from cu_aup.query import Query
from cu_api.core import set_headers
import pandas as pd
import pytz
from tqdm import tqdm

In [2]:
#| export
def _clean_company_data(company_data:list, #Returned company data as list of dictionaries,
                        Output_CF:list = [], # list of custom fields to include in export
                       ):
    """Process to clean returned company data"""

    core.prc_cf_dicts()

    custom_fields = getattr(config,'CUSTOM_FIELDS')
    custom_fields_dict = getattr(config,'CUSTOM_FIELDS_DICT')
    

    def clean_date(date):
        try:
            if date is None:
                return None  # Handle None values (empty rows)

            if isinstance(date, int) and len(str(date)) >= 6 and len(str(date)) <= 10:
                # If it's an integer with <= 10 digits, assume it's a Unix timestamp
                ct_timezone = pytz.timezone('US/Central')
                new_date = pd.to_datetime(date, unit='s',utc=True).tz_convert(ct_timezone)
                return new_date
            else:
                return date
        except Exception as e:
            return date

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items: output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)
        data_type = custom_fields[item_id].get('data_type')

        if item_name not in Output_CF or item_name is None:
            continue
        elif item_name is not None and ('options' in list(custom_fields[item_id].keys())):
            item_value = dict_item['value']
            value_name = core.cf_option_name(item_id,item_value)
            output_dict[item_name] = value_name
        elif data_type == 'Date':
            cleaned_date = clean_date(dict_item['value'])
            output_dict[item_name] = cleaned_date
        else:
            item_value = dict_item['value']
            output_dict[item_name] = item_value

    return output_dict

In [7]:
#| export
def search(search_query, # Instance of Query object
           clean_data:bool = True, #Whether to clean results or not
           drop_cols:list = None, # Columns to drop
          )->pd.DataFrame:
    """Get Company 
    Function to search copper companies, and return the resulting data.

    Parameters:      
      search_query: Instance of Query onject

        Query objects are used to house and process the query parameters for native fields to copper and 
        custom fields. 

            Info: help(Query) or ?Query
      
      clean_data: bool
        Whether data is cleaned prior to return
    
      drop_cols: list
        list of columns to drop, not include, in the returned dataframe. Names must match the copper field names exactly.
    
    Returns:
      pandas.
    """

    combined_results = core._search_loop(search_query,'https://api.copper.com/developer_api/v1/tasks/search')

    if not combined_results: pass
    if not clean_data: return combined_results
   
    if search_query._custom_fields: Output_CF = Query._custom_fields
    else:                           Output_CF = None

    cleaned_data = [_clean_company_data(result, Output_CF) for result in combined_results]
    cleaned_data_df = pd.DataFrame(cleaned_data)

    if drop_cols:      return cleaned_data_df.drop(columns=drop_cols,inplace=True)
    else:              return cleaned_data_df 

### Searching Copper Companies

Working with Copper Companies is a core feature of cuapi_wrapper and is done with the `companies.search` function.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()